In [3]:
from dotenv import load_dotenv 
from openai import OpenAI
load_dotenv()
client = OpenAI()

## text generation/chat

In [4]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "which city is China's Capital?"
        }
    ]
)

In [5]:
print(completion.choices[0].message)

ChatCompletionMessage(content='The capital of China is Beijing.', refusal=None, role='assistant', function_call=None, tool_calls=None)


### Returned object

- Type of `ChatCompletion`
- https://platform.openai.com/docs/api-reference/chat/object

In [13]:
type(completion)

openai.types.chat.chat_completion.ChatCompletion

In [15]:
from testUtils import print_properties
print_properties(completion)

id:
  chatcmpl-AJDBs7ryJnfpfJDLg6xg2F9FFn0RD
choices:
  [0]:
    finish_reason:
      stop
    index:
      0
    logprobs:
      None
    message:
      content:
        The capital of China is Beijing.
      refusal:
        None
      role:
        assistant
      function_call:
        None
      tool_calls:
        None
created:
  1729142300
model:
  gpt-4o-mini-2024-07-18
object:
  chat.completion
service_tier:
  None
system_fingerprint:
  fp_e2bde53e6e
usage:
  completion_tokens:
    7
  prompt_tokens:
    23
  total_tokens:
    30
  completion_tokens_details:
    audio_tokens:
      None
    reasoning_tokens:
      0
  prompt_tokens_details:
    audio_tokens:
      None
    cached_tokens:
      0
_request_id:
  req_ed3d3ebdbc6c119a6eb99185f09b34fd


In [9]:
import json
completion_dict = completion.to_dict()
print(json.dumps(completion_dict, indent=2))

{
  "id": "chatcmpl-AJDBs7ryJnfpfJDLg6xg2F9FFn0RD",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The capital of China is Beijing.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1729142300,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_e2bde53e6e",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 23,
    "total_tokens": 30,
    "completion_tokens_details": {
      "reasoning_tokens": 0
    },
    "prompt_tokens_details": {
      "cached_tokens": 0
    }
  }
}
